<a href="https://colab.research.google.com/github/MarcelaOlarte/Trabajo-de-acompa-amiento/blob/main/Acompa%C3%B1amiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# URL del dataset
url_json = "https://www.datos.gov.co/resource/4iz7-suhz.json"

# Lista de enfermedades que vamos a analizar
enfermedades = [
    "artritis", "diabetes", "hipertensi_n", "epoc", "asma",
    "insuficiencia_cardiaca", "c_ncer"
]

# Cambiamos valores 1/2 a Sí/No
df[enfermedades] = df[enfermedades].replace({1: "Sí", 2: "No"})

# Unimos datos en formato largo (melt)
df_melt = df.melt(
    id_vars=["sexo"],
    value_vars=enfermedades,
    var_name="Enfermedad",
    value_name="Respuesta"
)

# Conteo por Sexo, Enfermedad y Respuesta
tabla_resumen = df_melt.groupby(["sexo", "Enfermedad", "Respuesta"]).size().reset_index(name="Cantidad")

# Pivot para que Sí y No queden en columnas separadas
tabla_pivot = tabla_resumen.pivot_table(
    index=["sexo", "Enfermedad"],
    columns="Respuesta",
    values="Cantidad",
    fill_value=0
).reset_index()

tabla_pivot_limpia = tabla_pivot[tabla_pivot["sexo"].isin(["FEMENINO", "MASCULINO"])]
print(tabla_pivot_limpia)

In [ ]:
import pandas as pd

# Cargar datos
url_json = "https://www.datos.gov.co/resource/4iz7-suhz.json"
df = pd.read_json(url_json)

# Función para clasificar riesgo
def clasificar_riesgo(row):
    if row['hipertensi_n'] == 1 or row['diabetes'] == 1:
        return 'ALTO'
    else:
        return 'BAJO'

df['riesgo_cardiovascular'] = df.apply(clasificar_riesgo, axis=1)

# Contar por sexo y riesgo
conteo_riesgo_sexo = df.groupby(['sexo', 'riesgo_cardiovascular']).size().reset_index(name='conteo')

print(conteo_riesgo_sexo)

In [ ]:
# Filtrar personas mayores de 60 años con hipertensión (hipertensi_n == 1)
mayores_hipertensos = df[
    (df['grupo_etareo'] == '60 Y MAS') &
    (df['hipertensi_n'] == 1)
]

mayores_hipertensos

In [ ]:
# Columnas de enfermedades
enfermedades_cols = ['artritis','diabetes','hipertensi_n','epoc','asma','insuficiencia_cardiaca',
                     'c_ncer','huerfanas_hemofilias_y_otras','cirugia_cardiaca','trasplantados']

# Convertir 2 → 0 y 1 → 1 para poder operar
df_enfermedades = df[enfermedades_cols].replace({2:0, 1:1})

# Crear columna total de enfermedades por persona
df['total_enfermedades'] = df_enfermedades.sum(axis=1)

# Agrupar por sexo y grupo_etareo
df_agrupado = df.groupby(['sexo','grupo_etareo']).agg(
    n_personas=('total_enfermedades','count'),
    promedio_enfermedades=('total_enfermedades','mean'),
    total_enfermedades=('total_enfermedades','sum')
).reset_index()

# Redondear promedio
df_agrupado['promedio_enfermedades'] = df_agrupado['promedio_enfermedades'].round(2)

# Mostrar resultados
df_agrupado

In [ ]:
# Lista de columnas de enfermedades
cols_enfermedades = [
    'artritis', 'diabetes', 'hipertensi_n', 'epoc', 'asma',
    'insuficiencia_cardiaca', 'c_ncer', 'huerfanas_hemofilias_y_otras',
    'cirugia_cardiaca', 'trasplantados'
]

# Contar cuántas personas tienen cada enfermedad (valor 1 = Sí)
conteo_enfermedades = df[cols_enfermedades].apply(lambda x: (x == 1).sum()).sort_values(ascending=False)

# Convertir a DataFrame para ver mejor
tabla_enfermedades_ordenada = conteo_enfermedades.reset_index()
tabla_enfermedades_ordenada.columns = ['Enfermedad', 'Numero_personas']

tabla_enfermedades_ordenada

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Contar por sexo y riesgo cardiovascular
conteo_riesgo_sexo = df.groupby(['sexo', 'riesgo_cardiovascular']).size().reset_index(name='conteo')

# Gráfico de barras
plt.figure(figsize=(8,5))
sns.barplot(data=conteo_riesgo_sexo,
            x='sexo',
            y='conteo',
            hue='riesgo_cardiovascular',
            palette='Set2')

plt.title("Riesgo cardiovascular por sexo")
plt.ylabel("Número de personas")
plt.xlabel("Sexo")
plt.legend(title="Riesgo")
plt.show()


In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Cargar datos desde la URL
url_json = "https://www.datos.gov.co/resource/4iz7-suhz.json"
df = pd.read_json(url_json)

# Normalizar columna 'sexo'
df['sexo'] = df['sexo'].str.strip().str.upper()

# Lista de columnas de enfermedades
enfermedades_cols = [
    'artritis','diabetes','hipertensi_n','epoc','asma',
    'insuficiencia_cardiaca','c_ncer','huerfanas_hemofilias_y_otras',
    'cirugia_cardiaca','trasplantados'
]

# Convertir a 1 = Sí, 0 = No
for col in enfermedades_cols:
    df[col] = df[col].apply(lambda x: 1 if x == 1 else 0)

# Sumar por sexo
df_enf_sexo = df.groupby('sexo', dropna=False)[enfermedades_cols].sum().reset_index()

# Pasar a formato largo para Seaborn
df_enf_long = df_enf_sexo.melt(
    id_vars='sexo',
    value_vars=enfermedades_cols,
    var_name='Enfermedad',
    value_name='Número de personas'
)

# Gráfico de barras
plt.figure(figsize=(12,6))
sns.barplot(
    data=df_enf_long,
    x='Enfermedad',
    y='Número de personas',
    hue='sexo',
     palette=['#6a0dad', '#2ecc71', '#e74c3c']  # morado, verde, rojo
)
plt.title("Distribución de enfermedades por sexo")
plt.ylabel("Número de personas")
plt.xlabel("Enfermedad")
plt.xticks(rotation=45)
plt.legend(title="Sexo")
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Lista de columnas de enfermedades
enfermedades_cols = ['artritis','diabetes','hipertensi_n','epoc','asma',
                     'insuficiencia_cardiaca','c_ncer','huerfanas_hemofilias_y_otras',
                     'cirugia_cardiaca','trasplantados']

# Copiar base original
df_ciclo = df.copy()

# Convertir enfermedades a 1 = Sí, 0 = No
for col in enfermedades_cols:
    df_ciclo[col] = df_ciclo[col].apply(lambda x: 1 if x==1 else 0)

# Crear columna 'total_enfermedades' sumando todas las enfermedades
df_ciclo['total_enfermedades'] = df_ciclo[enfermedades_cols].sum(axis=1)

# Estadística descriptiva por ciclo de vida
stats_ciclo = df_ciclo.groupby('ciclo_de_vida').agg(
    n_personas=('total_enfermedades','count'),
    promedio_enfermedades=('total_enfermedades','mean'),
    total_enfermedades=('total_enfermedades','sum')
).reset_index()

print(stats_ciclo)

# Gráfico: Boxplot de total de enfermedades por ciclo de vida
plt.figure(figsize=(10,6))
sns.boxplot(data=df_ciclo, x='ciclo_de_vida', y='total_enfermedades', palette='Set2')
plt.title("Distribución de total de enfermedades por ciclo de vida")
plt.xlabel("Ciclo de vida")
plt.ylabel("Total de enfermedades por persona")
plt.xticks(rotation=45)
plt.show()

# Alternativa: Gráfico de barras con promedio de enfermedades por ciclo de vida
plt.figure(figsize=(10,6))
sns.barplot(data=stats_ciclo, x='ciclo_de_vida', y='promedio_enfermedades', palette='Set3')
plt.title("Promedio de enfermedades por ciclo de vida")
plt.xlabel("Ciclo de vida")
plt.ylabel("Promedio de enfermedades")
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Lista de columnas de enfermedades
enfermedades_cols = ['artritis','diabetes','hipertensi_n','epoc','asma',
                     'insuficiencia_cardiaca','c_ncer','huerfanas_hemofilias_y_otras',
                     'cirugia_cardiaca','trasplantados']

# Convertir 1 = Sí, 2 = No → 1 y 0
df_calc = df.copy()
for col in enfermedades_cols:
    df_calc[col] = df_calc[col].apply(lambda x: 1 if x==1 else 0)

# Agrupar por sexo
resumen_sexo = df_calc.groupby('sexo')[enfermedades_cols].agg(
    total_personas=('artritis', 'count'),  # número total de registros por sexo
    **{col+'_total': (col, 'sum') for col in enfermedades_cols},
    **{col+'_promedio': (col, 'mean') for col in enfermedades_cols}
).reset_index()

# Redondear promedios
for col in enfermedades_cols:
    resumen_sexo[col+'_promedio'] = resumen_sexo[col+'_promedio'].round(2)

resumen_sexo